In [26]:
# Install dependencies
!pip install -q amplpy
# -- IMPORTED --
import csv
import math
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Google Colab & Kaggle integration
MODULES=['ampl', 'coin', 'highs', 'gokestrel']
from amplpy import AMPL, Environment, tools


# Random Bit flip generation for initial solution
def bitFlipWarehouse(K, nWarehouses, warehousesData):
    contOpened = 0
    contClosed = 0 
    warehouseDisponibility = []
    for i in range(nWarehouses):
        #Si el costo del almacen es 0, significa que debe ser el principal. 
        #Sino, cualquier otro almacen
        if(warehousesData[i][1] == 0): 
            warehouseDisponibility.append(1)
            contOpened += 1
        else:
            randBit = random.randint(0,1)
            if(randBit == 1):
                if(contOpened < K):
                    warehouseDisponibility.append(1)
                    contOpened += 1
                else: 
                    warehouseDisponibility.append(0)
                    contClosed += 1         
            else: 
                if(contClosed < (nWarehouses - K)):
                    warehouseDisponibility.append(0)
                    contClosed += 1
                else:
                    warehouseDisponibility.append(1)
                    contOpened += 1
                    
    return warehouseDisponibility

# Lectura del archivo
file = "cap41.txt"
settings = []
with open(file) as rf:    
	rf = csv.reader(rf)    
	for row in rf:        
		settings.append(row[0])        
#print("datos",settings)
# Setting principal de cantidad de almacenes y clientes de la primera fila
nWarehouses = int(settings[0].split()[0])
nCustomers = int(settings[0].split()[1])
print(nWarehouses)
print(nCustomers)

warehouses = []
# Lectura de primer y segundo valor después de la lectura de la primera fila hasta nWarehouses.
for i in range(1, nWarehouses+1):
    capacityWarehouse= int(settings[i].split()[0])
    fixedCost = settings[i].split()[1]
    fixedCost = int(fixedCost.replace('.', ''))
    fixedWarehouseCost = fixedCost
    warehouses.append((capacityWarehouse, fixedCost))
#print("Capacity of Warehouses:",capacityWarehouse)
#print("Fixed Warehouses Cost:", fixedWarehouseCost)
print(pd.DataFrame(warehouses).T)


nOfIterations = 1
totalCost = 0
X = []
K = 8
    
for n in range(nOfIterations):
    assignmentCost = 0    
    X = bitFlipWarehouse(K, nWarehouses, warehouses)
print(X)
np.savetxt("./AMPL/X.txt", X, delimiter=',', fmt='%i')

16
50
     0     1     2     3     4     5     6     7     8     9     10    11  \
0  5000  5000  5000  5000  5000  5000  5000  5000  5000  5000  5000  5000   
1  7500  7500  7500  7500  7500  7500  7500  7500  7500  7500     0  7500   

     12    13    14    15  
0  5000  5000  5000  5000  
1  7500  7500  7500  7500  
[0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0]


In [27]:
ampl = AMPL(Environment(r'C:\Users\Tekan\Documents\CFLP_IOA\AMPL\ampl_mswin64'))
ampl.option['solver'] = 'gurobi'

In [ ]:
#Define a time limit for checking if initial solution is factible, else, something is wrong
if(nWarehouses <= 500):
    timeout = 60 * 5
elif(nWarehouses <= 1000):
    timeout = 60 * 10
else: timeout = 60 * 25
timeout_start = time.time()
finished = False
timePassed = False
while not(finished or timePassed):
    #if statement for AMPL to tell that X is factible
    if(time.time() > timeout_start + timeout): timePassed = True
    #if X is not factible, then initial solution goes for another bitFlip
    X = bitFlipWarehouse(K, nWarehouses, warehouses)
    nOfIterations += 1
    
if(timePassed): print("Time limit exceeded, try changing K variable")
if(finished): 
    print("Initial solution factible after", nOfIterations,"iterations!")
    print("initial cost saved in 'initialCost.txt', please run main.py")
    np.savetxt("initialCost.txt", totalCost, fmt='%f')